<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part 2, version 0.1,  June 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![title](images/header.png)

# II - SBAE Time-Series Extraction & Change Detection
### Extract various time-series outputs for point data from Google Earth Engine
-------

This notebook takes you through the process of extracting outputs from various time-series change detection algorithms and structure them in a so-called data-frame (e.g. tabular structure).

### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

Styles()

/home/vollrath/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [2]:
# Area of Interest

# country/province case example
country = 'Buhweju'   
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME', country)) # here any feature collection can be selected

# Asset case
#aoi = ee.FeatureCollection("my_aoi_feature_collection")

# Point Feature Collection from where to extract points
fc = ee.FeatureCollection("users/andreasvollrath/01_test_sbae_points").filterBounds(aoi)

# the column of a unique point identifier in your dataset
point_id_name = 'point_id'

grid_size = 0.25 # that's the size of the grid we are parallelizing on in degrees
workers = 10 # number of parallel EE requests

#### 2b - Check inputs visually

In [3]:
# create the grid
grid, grid_fc = h.processing_grid(aoi, grid_size, grid_size)

Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(grid_fc)
Map.addLayer(fc)
Map.addLayer(aoi)
Map

SepalMap(center=[-0.3203418090924264, 30.347812392179087], controls=(ZoomControl(options=['position', 'zoom_in…

#### 2c - Time-series parameter settings

In [4]:
# start of calibration period (mainly for bfast)
start_calibration = "2016-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor = "2018-01-01"  # YYYY-MM-DD format
end_monitor =  "2020-01-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = 'results_new'  # goes to module_results/sbae_point_analysis if left to None

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = False
landtrendr = True
global_products = False

# select the bands to extract
bands=['green', 'red', 'nir', 'swir1', 'swir2', 'ndfi'] # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness

# select the band for univariate ts-analysis (has to be inside bands list)
ts_band = 'ndfi'

# select the resolution to which the satellite data will be resized.
scale=100  # in meters

### DO NOT CHANGE YET ###
satellite='Landsat'
max_cloud_cover = 75  # in percentage (0-100)

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [5]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'
}

# cusum parameters
cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

# slope parameters
bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

# time-series metrics
ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': False,
    'z_threshhold': 3
}

# ccdc parameters
ccdc_params = {
    'run': ccdc,
}

landtrendr_params = { 
        'run': landtrendr,
        'maxSegments':            6,
        'spikeThreshold':         0.9,
        'vertexCountOvershoot':   3,
        'preventOneYearRecovery': True,
        'recoveryThreshold':      0.25,
        'pvalThreshold':          0.05,
        'bestModelProportion':    0.75,
        'minObservationsNeeded':  3
}

# global products parameters
global_products = {
    'run': global_products,
    'gfc': True,  # will include tree-cover 2000, loss, gain, lossyear
    'tmf': True,  # will include deforestation and degradation year for tropical moist forests
    'tmf_years': True, # will include classes per year - according to the monitor period
    'esa_lc20': True, # will include ESA LandCover Product class
    'copernicus_lc': True, # will include ESA LandCover Product class - acording to the monitoring years
    'esri_lc': True,  # will include the classes from ESRI World Cover 2020
    'lang_tree_height': True, # returns the Tree Height from Lang et al 2022
    'potapov_tree_height': True, # returns the tree height from Potapov et al. 2019 
    'dynamic_world_tree_prob': True, # returns Min, Max, Mean and StdDev of the trees probability for the monitoring period
    'dynamic_world_class_mode': True, # returns the mode of the class for the monitoring period
    'elevation': True  # returns elevation, slope and aspect
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': workers,
    'ts_params': {
        'start_calibration': start_calibration,
        'start_monitor': start_monitor,
        'end_monitor': end_monitor,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'bands': bands,
        'ts_band': ts_band,
        'satellite': satellite,
        'scale': scale,
        'max_cc': max_cloud_cover,
        'outlier_removal': True,
        'smooth_ts': True
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'landtrendr_params': landtrendr_params,
    'global_products': global_products
}

### 4 - Run the time-series data extraction

**Execute only**

In [6]:
h.get_change_data(aoi, fc, config_dict)

 Parallelizing time-series extraction on 10 threads for a total of 4 grid cells.
 Processing gridcell 1
 Processing gridcell 3
 Grid cell 0 does not contain any points. Going on with next grid cell.
 Processing gridcell 2


2022-07-21 21:57:21.758878: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-21 21:57:21.758930: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ddb8eaef59cb): /proc/driver/nvidia/version does not exist
2022-07-21 21:57:21.759777: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 Grid cell 1 with 22 points done in: 0:01:02.652490
 Grid cell 2 with 1 points done in: 0:01:16.133091
 Grid cell 3 with 4 points done in: 0:01:44.042500


/home/vollrath/.local/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Deleting temporary files
 Processing has been finished successfully. Check for final_results files in your output directory.


In [7]:
import pandas as pd
pd.read_pickle('results_new/results_Landsat_ndfi_2016-01-01_2018-01-01_2020-01-01_0.25.pickle')

,point_idx,point_id,dates,ts,images,geometry,ltr_magnitude,ltr_dur,ltr_yod,ltr_rate,...,cusum_confidence,cusum_magnitude,ts_mean,ts_sd,ts_min,ts_max,bs_slope_mean,bs_slope_sd,bs_slope_max,bs_slope_min
0,0,18,"DatetimeIndex(['2016-01-03', '2016-02-04', '20...","{'green': [355.0, 409.5, 454.3333333333333, 47...",62,POINT (30.31263 -0.20034),0.000000,3,2016,0.000000,...,0.180407,966.455078,9864.350000,133.974590,9581.000000,10000.000000,34.502657,35.467765,136.405646,-79.073488
1,0,33,"DatetimeIndex(['2016-02-05', '2016-02-29', '20...","{'green': [668.0, 909.5, 618.0, 619.5, 744.0, ...",43,POINT (30.46809 -0.39630),-1251.350830,3,2016,-417.116943,...,0.105496,5305.422852,6724.113636,931.995258,4427.500000,8195.000000,341.879465,388.317726,2044.907904,-989.087841
2,1,34,"DatetimeIndex(['2016-01-28', '2016-02-05', '20...","{'green': [1430.0, 1112.5, 875.0, 871.0, 857.3...",48,POINT (30.46809 -0.35711),0.000000,3,2016,0.000000,...,0.069203,10996.112305,1324.440805,1719.072924,-2067.500000,4697.666667,-539.446458,434.610352,733.944933,-2004.097955
3,2,35,"DatetimeIndex(['2016-01-28', '2016-02-05', '20...","{'green': [780.0, 704.0, 641.0, 638.0, 771.0, ...",38,POINT (30.46809 -0.31792),-4087.484619,3,2016,-1362.494873,...,0.415903,7726.250000,7951.231746,946.734378,6326.000000,10000.000000,1301.412831,287.806516,2454.871763,371.550041
4,3,36,"DatetimeIndex(['2016-01-12', '2016-02-05', '20...","{'green': [1473.0, 1125.0, 1099.0, 940.0, 783....",52,POINT (30.46809 -0.27872),0.000000,3,2016,0.000000,...,0.335445,20720.392578,2704.688810,2323.353809,-1794.800000,7636.500000,1748.271630,488.830590,3336.634910,344.684770
5,0,12,"DatetimeIndex(['2016-01-27', '2016-02-28', '20...","{'green': [599.0, 532.5, 528.0, 610.0, 551.0, ...",53,POINT (30.31263 -0.43550),-1019.547363,3,2016,-339.849121,...,0.235329,843.465820,9934.404321,115.534742,9523.750000,10000.000000,74.167793,21.054429,151.574144,4.938620
6,1,13,"DatetimeIndex(['2016-01-03', '2016-01-27', '20...","{'green': [578.0, 715.0, 683.6666666666666, 65...",57,POINT (30.31263 -0.39630),0.000000,3,2016,0.000000,...,0.381074,11025.591797,5500.151306,1121.514038,3572.750000,7176.000000,1051.312089,245.025782,1886.178195,270.782718
7,2,14,"DatetimeIndex(['2016-01-03', '2016-01-27', '20...","{'green': [567.0, 611.5, 601.6666666666666, 61...",56,POINT (30.31263 -0.35711),0.000000,3,2016,0.000000,...,0.514554,13236.300781,8239.605172,1097.975695,6305.666667,9574.000000,1628.308144,120.950768,1933.057163,1159.935966
8,3,15,"DatetimeIndex(['2016-01-03', '2016-01-27', '20...","{'green': [445.0, 526.5, 484.6666666666667, 60...",54,POINT (30.31263 -0.31792),0.000000,3,2016,0.000000,...,0.407868,10793.342773,8433.293103,1097.817143,6610.666667,9926.000000,712.560051,261.746319,1750.012289,-33.741880
9,4,16,"DatetimeIndex(['2016-01-03', '2016-01-27', '20...","{'green': [558.0, 650.0, 653.3333333333334, 66...",62,POINT (30.31263 -0.27872),-1241.039355,3,2016,-413.679785,...,0.227826,7350.157715,8507.994253,953.247212,6366.000000,9939.000000,782.468911,189.098032,1526.588371,160.684237
